In [ ]:
!pip install pandas numpy sentence-transformers faiss-cpu transformers torch openpyxl

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
# from transformers import pipeline
import torch

# Load Excel file
df = pd.read_excel('equipments.xlsx')

# Optional: Chunk data if needed
chunks = df.apply(lambda row: row.to_string(), axis=1).tolist()
# print(chunks)

# Generate unique IDs for each chunk
ids = [f"doc_{i}" for i in range(len(chunks))]

# Initialize embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(chunks)

# Convert embeddings to numpy array
embeddings_np = np.array(embeddings).astype('float32')

# Create FAISS index
d = embeddings_np.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings_np)

# Save the FAISS index
faiss.write_index(index, 'faiss_index.bin')

# Store metadata (chunks and IDs)
metadata = {'chunks': chunks, 'ids': ids}
import pickle
with open('metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

# Embed user question
user_question = "List all equipments in Telemetry."
question_embedding = model.encode([user_question]).astype('float32')

# Perform vector search
k = 30  # Number of results to retrieve
distances, indices = index.search(question_embedding, k)

# Load metadata
with open('metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

retrieved_content = [metadata['chunks'][i] for i in indices[0]]

# Extract metadata from search results
retrieved_text = " ".join([str(content) for content in retrieved_content])

# Chunk the retrieved text to fit model's token limit
chunk_size = 1024  # Adjust based on model's token limit
chunks = [retrieved_text[i:i+chunk_size] for i in range(0, len(retrieved_text), chunk_size)]

# Prepare input for LLM
input_text = f"Question: {user_question}\nContext: {retrieved_text}"

system_role = "You will receive the user's question along with the search results of that question over a database. The database is an equipments record and contains columns: SN =Serial No., slno =Equipment Serial Number, eqID= equipment ID, eqName = equipment Name, loc = Location of equipment, srvNo = Stores reference number, fails = number of times the equipment failed. Give the user the proper answer. Also display all the data input you have from the admin side."
prompt = f"System: {system_role}\nUser: {input_text}"


# # Prepare input for LLM
# input_text = f"Question: {user_question}\nContext: {chunks[0]}"  # Start with the first chunk

# # Use Hugging Face transformer model for response generation
# generator = pipeline('text-generation', model='distilgpt2', device=0 if torch.cuda.is_available() else -1)

# Generate response for the first chunk
# response = generator(input_text, max_length=512, num_return_sequences=1)

# # Print the complete response
# complete_response = response[0]['generated_text']
# print(complete_response)

# # Process remaining chunks if needed
# for chunk in chunks[1:]:
#     input_text = f"Context: {chunk}"
#     response = generator(input_text, max_length=512, num_return_sequences=1)
#     complete_response += response[0]['generated_text']

# print(complete_response)


import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('gemini')

genai.configure(api_key=GOOGLE_API_KEY)

modelX = genai.GenerativeModel('gemini-1.5-pro-latest')
response = modelX.generate_content(prompt, stream=True)

for chunk in response:
  print(chunk.text)

# Initialize an empty string to hold the complete response
complete_response = ""

# Concatenate all chunks into the complete response string
for chunk in response:
    complete_response += chunk.text

# Print the complete response
print(complete_response)


System
:
```json
{
  "question": "List all equipments in
 Telemetry.",
  "database": [
    {
      "SN
": "35",
      "slno": "INA308X24M",
      "eqID": "NOP///UHF
/TX",
      "eqName": "VHF Computer HP Compaq pro 6300",
      "loc": "TELEMETRY
",
      "srvNo": "-",
      "fails": "0"
    },
    {
      "SN": "28",
      "slno": "NaN",
      "eqID": "NOP
///RKVM/04",
      "eqName": "RKVM",
      "loc": "TELEMETRY",
      "srvNo": "-",
      "fails": "0"
    },
    {

      "SN": "33",
      "slno": "NaN",
      "eqID": "Station GW2",
      "eqName": "DELL Precision 5820",
      "loc": "TELEMETRY",
      "srvNo": "-",
      "
fails": "0"
    },
    {
      "SN": "34",
      "slno": "71954Y1",
      "eqID": "NOP///UHF/RX",
      "eqName": "UHF Computer DELL-T56
00",
      "loc": "TELEMETRY",
      "srvNo": "-",
      "fails": "0"
    },
    {
      "SN": "32",
      "slno": "NaN",
      "eqID": "Station GW1",

      "eqName": "DELL Precision 5820",
      "loc": "TELEMETRY",
      "srv